<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamC/blob/main/Code/ETL2Datawarehouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kursarbeit Datenmanagement und -Archivierung WS 23/24**

Master-Code Name-CALICO:MA


**Erstellung der Datenbank**


*   Datenbankinitialisierung:



> Laden der benötigten Libraries

In [ ]:
import os
import pandas as pd
from functools import reduce
import sqlite3
from sqlite3 import Error
import csv
#import requests

>Google drive verbinden

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!ls drive/

Mounted at /content/drive
MyDrive  Shareddrives


>Clonen des Repository (https://github.com/Fuenfgeld/DMA2024TeamC.git) in den Ordner calico_ma_repo im google drive. Hier bei wird das Programm git installiert. Soll.te dies zum ersten mal ausgeführt werden wird das komplette Repository geklont. Achtung: Sollte das Repository bereits früher in diesen Unterordner geclont worden sein, so wird ein "git fetch" ausgeführt und die aktuellsten Daten aus dem Repository weredn heruntergeladen. Dabei weredn alle Daten überschreiben und nicht mehr vorhandene Daten im Repository werden acuh im google drive gelöscht.

In [ ]:
#definieren des Ordners in dem die repo gesichert werdern soll
google_drive_folder = "/content/drive/MyDrive"
repo_colab_name = "calico_ma_repo"
#full_path_repo_colab = f"{google_drive_folder}+"//"+{repo_colab_name}"
full_path_repo_colab = os.path.join(google_drive_folder,repo_colab_name)
if not os.path.exists(full_path_repo_colab):
    os.makedirs(full_path_repo_colab)
    print(f"Ordner {full_path_repo_colab} angelegt")
else:
    print(f"Ordner {full_path_repo_colab} existiert bereits")

In [ ]:
! apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


>Achtung: Herunterladen der Repo. Achtung es wird ein fetch ausgeführt und alle Daten aus dem repository übernommen. Es wird keine Abfrage gemacht, ob Daten überschrieben werden sollen.

In [ ]:
if os.path.exists(full_path_repo_colab):
    # If the repository already exists, navigate to it and pull the latest changes
    %cd $full_path_repo_colab
    # Fetch the remote repository
    !git fetch origin

    # Reset your local branch to match the remote branch
    !git reset --hard origin/main
else:
    # If the repository does not exist, clone it
    !git clone https://github.com/Fuenfgeld/DMA2024TeamC.git $full_path_repo_colab

In [ ]:
!ls $full_path_repo_colab

>Definition der Umgebnungsvariablen

>Verbindung zu CSV-Dateien herstellen und Kategorisierung nach der Tumorerkrankung
**AMENDMENT FOR ML: project_name = "CALICO_MA_ML"**

In [ ]:
#repo_home = "/home/gunnar/DMA2024TeamC/" ## muss entsprechend angepasst werden für das lokale repo verzeichnis
repo_home = full_path_repo_colab+"/" ## repo in google drive
csv_data_folder = repo_home + "Data/Quelldaten/CSV_Data/"
project_name = "CALICO_MA"
print(repo_home)
print (csv_data_folder)

> **Datebankinitialisierung**

> Erklärung des unten folgenden Codesb durch GitHub CoPilot

>The provided code is a Python script that establishes a connection to a SQLite database and creates a cursor object for executing SQL commands.
>The `create_connection_local` function is defined to establish a connection to a SQLite database. The function takes one parameter, `local_path`, which is the path to the database file. Inside the function, a `try` block is used to attempt to establish the connection. The `sqlite3.connect` function is called with the path to the database file, which is constructed by concatenating `local_path`, the string 'Data/CALICO_MA_Database/', and the global variable `project_name` with '.db' extension. If the connection is established successfully, the connection object is returned. However, the `print(sqlite3.version)` statement after the `return` statement will never be executed because the function exits as soon as it hits the `return` statement. If there is an error while establishing the connection, it is caught by the `except` block, and a message is printed to the console.
>After the `create_connection_local` function is defined, it is called with `repo_home` as the argument, and the returned connection object is stored in the `conn` variable.
>Next, a cursor object is created by calling the `cursor` method on the connection object. This cursor object, stored in the `cur` variable, can be used to execute SQL commands.
>Finally, a success message is printed to the console, indicating that the connection to the SQLite database was established successfully. The message includes the path to the database file.

In [ ]:
def create_connection_local(local_path):
  conn = None;
  try:
    #Establishing the connection
    conn = sqlite3.connect(local_path+'Data/CALICO_MA_Database/'+project_name+'.db')
    return conn
    print(sqlite3.version)
  except Error as e:
    print("Error while connecting to sqlite", e)
conn = create_connection_local(repo_home)
# Creating a cursor object using the cursor() method
cur = conn.cursor()
print("Successfully Connected to SQLite Public Data Warehouse: "+repo_home+'Data/CALICO_MA_Database/'+project_name+'.db')


> Datenimport aus den CSV Quelldateien in Pandas Dataframes 


Folgende Schritte werden dabei im Code durchgeführt:

This Python script is used to read multiple CSV files from different directories, create pandas DataFrames from them, and then concatenate these DataFrames based on their types.

Here's a step-by-step explanation:

1. Two lists are defined: `csvs` and `tumortypes`. `csvs` contains the names of different types of CSV files, and `tumortypes` contains the names of different types of tumors.

2. An empty dictionary `df` is created to store the DataFrames.

3. The script then enters a nested loop where it iterates over each tumor type and each CSV type. For each combination, it constructs a key (e.g., "procedures_Breast_Cancer") and a file path.

4. It reads the CSV file at the constructed path into a DataFrame, adds a new column `cancer_type` to the DataFrame, and stores the DataFrame in the `df` dictionary with the constructed key.

5. The DataFrame is also appended to a list in the `csv_dfs` dictionary corresponding to its CSV type.

6. After all CSV files have been read and their DataFrames stored, the script concatenates the DataFrames in each list in the `csv_dfs` dictionary. The result is a dictionary where each key is a CSV type and each value is a DataFrame that is the concatenation of all DataFrames of that CSV type.

7. Finally, the script prints the first few rows of each DataFrame in the `df` and `csv_dfs` dictionaries.

In [ ]:
# Generate dataframes and add cancer_type information as a new column

csvs = ["procedures", "encounters", "immunizations", "medications", "observations", "patients"]

tumortypes = ["Breast_Cancer", "Lung_Cancer", "Colorectal_Cancer"]

df = {}
csv_dfs = {csv: [] for csv in csvs}

for tumor in tumortypes:
    for csv in csvs:
        key = f"{csv}_{tumor}"
        path = f"{csv_data_folder}{tumor}/{csv}.csv"
        df[key] = pd.read_csv(path, sep=",")
        df[key] = df[key].assign(cancer_type=tumor)

        # Add the dataframe to the list for this csv
        csv_dfs[csv].append(df[key])

# Check the sum of the number of rows in the original dataframes for each csv
for csv in csvs:
    total_rows = sum(len(df[csv + '_' + tumor]) for tumor in tumortypes)
    print(f"Total number of rows in original dataframes for {csv}: {total_rows}")
    print("\n")
# Concatenate the dataframes for each csv
for csv in csvs:
    csv_dfs[csv] = pd.concat(csv_dfs[csv])

# Check the number of rows and unique cancer types in each concatenated dataframe
for csv, df in csv_dfs.items():
    print(f"Concatenated DataFrame for {csv}:")
    print(f"Number of rows: {len(df)}")
    print(f"Unique cancer types: {df['cancer_type'].unique()}")
    print("\n")


>Quality checks of the dataframes

In [ ]:
# .nunique(axis=0) returns the number of unique values for each columns in the concanated dataframes.
for csv, df in csv_dfs.items():
    print(f"Quality for concatenated DataFrame {csv}:\n{df.nunique(axis=0)}")
    print("\n")



In [ ]:
#percentage of not null values in each column in the concanated dataframes.
for csv, df in csv_dfs.items():
    print(f"percentage of not null values in each column of concatenated DataFrame {csv}:\n{df.count()/len(df) * 100}")
    print("\n")


In [ ]:
#Delete Tables in case they exist
cur.execute("DROP TABLE IF EXISTS procedures")
cur.execute("DROP TABLE IF EXISTS encounters")
cur.execute("DROP TABLE IF EXISTS immunizations")
cur.execute("DROP TABLE IF EXISTS medications")
cur.execute("DROP TABLE IF EXISTS observations")
cur.execute("DROP TABLE IF EXISTS patients")
cur.execute("DROP TABLE IF EXISTS facts_table")

>Create tables including facts_table
**AMENDMENTS FOR ML: COUNTY as STRING, ZIP as STRING**

In [ ]:
# Create tables including facts_table

sql_table = {}

create_table_command = """CREATE TABLE IF NOT EXISTS patients (
                           Id         STRING PRIMARY KEY,
                           BIRTHDATE  DATE,
                           DEATHDATE  DATE,
                           SSN        STRING,
                           DRIVERS    STRING,
                           PASSPORT   STRING,
                           PREFIX     STRING,
                           FIRST      STRING,
                           LAST       STRING,
                           SUFFIX     STRING,
                           MAIDEN     STRING,
                           MARITAL    STRING,
                           RACE       STRING,
                           ETHNICITY  STRING,
                           GENDER     STRING,
                           BIRTHPLACE STRING,
                           ADDRESS    STRING,
                           CITY       STRING,
                           STATE      STRING,
                           COUNTY     STRING,
                           ZIP        STRING,
                           LAT        DECIMAL(6,2),
                           LON        DECIMAL(6,2),
                           HEALTHCARE_EXPENSES DECIMAL(9,2),
                           HEALTHCARE_COVERAGE DECIMAL(9,2),
                           cancer_type STRING
  );

CREATE TABLE IF NOT EXISTS encounters (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ORGANIZATION STRING,
                           PROVIDER STRING,
                           PAYER STRING,
                           ENCOUNTERCLASS STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_ENCOUNTER_COST DECIMAL(9,2),
                           TOTAL_CLAIM_COST DECIMAL(9,2),
                           PAYER_COVERAGE DECIMAL(9,2),
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           cancer_type STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
);



CREATE TABLE IF NOT EXISTS observations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           cancer_type STRING,
                           FOREIGN KEY (PATIENT) REFERENCES patients (Id),
                           FOREIGN KEY (Encounter) REFERENCES encounters (Id)
);


CREATE TABLE IF NOT EXISTS medications (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           PAYER STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST DECIMAL(9,2),
                           PAYER_COVERAGE DECIMAL(9,2),
                           DISPENSES DECIMAL(9,2),
                           TOTALCOST DECIMAL(9,2),
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           cancer_type STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
  );

CREATE TABLE IF NOT EXISTS procedures (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST DECIMAL(9,2),
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           cancer_type STRING,
                           FOREIGN KEY (PATIENT) REFERENCES patients (Id),
                           FOREIGN KEY (Encounter) REFERENCES encounters (Id)
                       );

CREATE TABLE IF NOT EXISTS immunizations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST DECIMAL(9,2),
                           cancer_type STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
  );

CREATE TABLE IF NOT EXISTS facts_table (
                          cancer_type STRING,
                          patient_ID STRING,
                          encounter_ID STRING,
                          patient_LAT        DECIMAL(6,2),
                          patient_LON        DECIMAL(6,2),
                          patient_COUNTY     STRING,
                          patient_ZIP        STRING,
                          patient_HEALTHCARE_EXPENSES DECIMAL(9,2),
                          patient_HEALTHCARE_COVERAGE DECIMAL(9,2),
                          patient_BIRTHDATE DATE,
                          observations_CODE STRING,
                          observations_VALUE STRING,
                          observations_DATE  DATE,
                          observations_AGE DECIMAL(6,2),
                          procedures_BASE_COST DECIMAL(11,2),
                          medications_TOTALCOST DECIMAL(11,2),
                          immunizations_BASE_COST DECIMAL(11,2),
                          encounter_BASE_ENCOUNTER_COST DECIMAL(11,2),
                          FOREIGN KEY (patient_ID) REFERENCES patients (Id),
                          FOREIGN KEY (patient_ID) REFERENCES observations (PATIENT),
                          FOREIGN KEY (encounter_ID) REFERENCES observations (ENCOUNTER),
                          FOREIGN KEY (patient_ID) REFERENCES encounters (PATIENT),
                          FOREIGN KEY (encounter_ID) REFERENCES encounters (Id),
                          FOREIGN KEY (patient_ID) REFERENCES procedures (PATIENT),
                          FOREIGN KEY (encounter_ID) REFERENCES procedures (ENCOUNTER),
                          FOREIGN KEY (patient_ID) REFERENCES medications (PATIENT),
                          FOREIGN KEY (encounter_ID) REFERENCES medications (ENCOUNTER),
                          FOREIGN KEY (patient_ID) REFERENCES immunizations (PATIENT),
                          FOREIGN KEY (encounter_ID) REFERENCES immunizations (ENCOUNTER)

);
"""
# Execute the SQL command
cur.executescript(create_table_command)

conn.commit()

In [ ]:
#Insert data from the concanated data frames into the tables

for csv, df in csv_dfs.items():
    df.to_sql(csv, conn, if_exists='append', index=False)

conn.commit()

In [ ]:
#Check if data is in the tables. 10 random rows are selected from each table

for csv in csvs:
    cur.execute(f"SELECT * FROM {csv} ORDER BY RANDOM() LIMIT 10")
    rows = cur.fetchall()
    df_random = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
    print(f"SQL tables from {repo_home}Data/CALICO_MA_Database/{project_name}.db' \n\n{csv} {df_random} ")


### Die Daten werden aus den verschiedenen Tabellen extrahiert und ggf. transformiert
 >Die Daten aus **Patients** werden erstmal in die Faktentabelle übertragen. Es handelt sich um die Krebsart, die Patienten ID, die geographische Koordinaten, die Healthcarekosten, die vom Patient getragen werden und diejenigen, die von der Krankenversicherung bezahlt werden, und zum Ende das Geburtsdatum.
Damit die Faktentabelle keine Redundanzen aufweist, werden die Encounter ID nicht mehr einzeln gespeichert. Stattdessen werden in der Faktentabelle für jeden Patienten die gesamten Kosten gespeichert, die in jeder Kategorie (Encounters, Observations, Immunizations, Procedures umd Medications) verursacht wurden.  
Die Transaktion in der Faktentabelle der Datenbank wird mit commit beendet.
**AMENDMENT FOR ML: patient_COUNTY and patient_ZIP included**

ACHTUNG: 53% der Patienten haben keine ZIP



In [ ]:
# TEST how many patients undergo Procedures
#cur.execute('''SELECT COUNT(*) from procedures GROUP BY procedures.PATIENT''')
#records = cur.fetchall()
#i=1
#for row in records:
#  print(f'patient', i, 'undergoes ', row,' procedures')
#  i=i+1

In [ ]:
# Datenextraktion aus der Patientstabelle
cur.execute('''INSERT INTO facts_table
                    ( cancer_type, patient_ID, encounter_ID, patient_LAT, patient_LON, patient_COUNTY, patient_ZIP, patient_HEALTHCARE_EXPENSES , patient_HEALTHCARE_COVERAGE, patient_BIRTHDATE)
                    SELECT
                        p.cancer_type AS cancer_type,
                        p.Id AS patient_ID,
                        NULL AS encounter_ID,  -- Replace with the actual encounter_ID or set to NULL if not applicable
                        p.LAT AS patient_LAT,
                        p.LON AS patient_LON,
                        p.COUNTY AS patient_COUNTY,
                        p.ZIP AS patient_ZIP,
                        p.HEALTHCARE_EXPENSES AS patient_HEALTHCARE_EXPENSES,
                        p.HEALTHCARE_COVERAGE AS patient_HEALTHCARE_COVERAGE,
                        p.BIRTHDATE AS patient_BIRTHDATE
                    FROM patients p
                    ;''')

# Ende der Transaktion
conn.commit()

# Schnelle Überprüfung der Datenübertragung ins die Faktentabelle
cur.execute('''SELECT patient_ID, patient_COUNTY, patient_ZIP, patient_LON, patient_LAT, patient_HEALTHCARE_EXPENSES, patient_HEALTHCARE_COVERAGE, patient_BIRTHDATE FROM facts_table''')
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)

Die Daten werden aus **Observations** eintragen:
nur die Tuples mit dem neuesten Datum werden berücksichtigt. Gespeichert werden die Art der Observation, in unserem Fall **nur den QALY**, den neuesten Wert und das entsprechende Datum. Ausserdem wird das Alter am Tag der letzten Observation berechnet und eingetragen. Die Transaktion wird mit commit beendet. Das Speichern der neuen Einträge wird überprüft.

In [ ]:
# Datenextraktion aus der Observationstabelle
cur.execute('''UPDATE
                  facts_table
               SET
                  observations_CODE = obs.CODE,
                  observations_VALUE = obs.maxVALUE,
                  observations_DATE = obs.maxDATE,
                  observations_AGE = obs.maxDATE- patient_BIRTHDATE
               FROM
                    (SELECT PATIENT, CODE, MAX(VALUE) as maxVALUE, MAX(observations.DATE) as maxDATE
                    FROM observations
                    WHERE CODE LIKE "QALY"
                    GROUP BY PATIENT
                    ) AS obs
                WHERE
                  facts_table.patient_ID = obs.PATIENT
                    ;''')

# Ende der Transaktion
conn.commit()

# Schnelle Überprüfung der Datenübertragung ins die Faktentabelle
cur.execute('''SELECT patient_ID, observations_CODE, observations_VALUE, patient_BIRTHDATE, observations_DATE, observations_AGE  FROM facts_table ''')
records = cur.fetchall()
# show 10 rows
for row in records[:30]:
  print(row)

print("\n")


Die Daten werden aus **Encounters** eintragen. Es handelt sich um die Summe aller Encounter Kosten für jeden einzelnen Patienten.
Falls dieser Wert nicht existiert, wird an ihrer Stelle 0 eingetragen.
Jede Transaktion wird mit commt beendet. Das Speichern der neuen Einträge in die Faktentabelle wird überprüft.


In [ ]:
# Datenextraktion aus der Encounterstabelle und Berechnung der Summe aller Encounterkosten für jeden Patienten
cur.execute('''UPDATE
                  facts_table
               SET
                  encounter_Base_Encounter_Cost = enc.TOTALCOST
               FROM
                    (SELECT PATIENT, SUM(BASE_ENCOUNTER_COST) as TOTALCOST
                    FROM encounters, facts_table
                    WHERE facts_table.patient_ID = encounters.PATIENT
                    GROUP BY PATIENT
                    ) AS enc
                WHERE
                  facts_table.patient_ID = enc.PATIENT
                    ;''')

# Ende der Transaktion
conn.commit()

# Falls es keine Encounterkosten gibt, wird der Wert 0 stattdessen eingetragen
cur.execute('''UPDATE
                  facts_table
               SET
                  encounter_Base_Encounter_Cost = 0
                WHERE encounter_Base_Encounter_Cost IS NULL
                    ;''')

# Ende der Transaktion
conn.commit()

# Kurze Überprüfung
cur.execute('''SELECT patient_ID, encounter_BASE_ENCOUNTER_COST FROM facts_table''')
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)


In [ ]:
cur.execute('''SELECT patient_ID, encounter_BASE_ENCOUNTER_COST, SUM(BASE_ENCOUNTER_COST)
                FROM facts_table, encounters WHERE facts_table.patient_ID = encounters.PATIENT
                GROUP BY patient_ID''')
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)

Die Daten werden aus **Medications** eintragen. Es handelt sich um die Summe aller MedikationsKosten für jeden einzelnen Patienten.
Falls dieser Wert nicht existiert, wird an ihrer Stelle 0 eingetragen.
Jede Transaktion wird mit commt beendet. Das Speichern der neuen Einträge in die Faktentabelle wird überprüft.

In [ ]:
cur.execute('''     UPDATE facts_table SET medications_TotalCost = med.TOTALCOST
                    FROM
                    (SELECT PATIENT, SUM(Totalcost) as TOTALCOST
                    FROM medications, facts_table
                    WHERE facts_table.patient_ID = medications.PATIENT
                    GROUP BY PATIENT) as med
                    WHERE facts_table.patient_ID = med.PATIENT
                    ;''')
conn.commit()

In [ ]:
cur.execute('''UPDATE
                  facts_table
               SET
                  medications_TOTALCOST = 0
                WHERE medications_TOTALCOST IS NULL
                    ;''')
conn.commit()

In [ ]:
cur.execute('''SELECT patient_ID, medications_TOTALCOST, SUM(TOTALCOST)
                FROM facts_table, medications WHERE facts_table.patient_ID = medications.PATIENT
                GROUP BY patient_ID''')
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)

Die Daten werden aus **Procedures** eintragen. Es handelt sich um die Summe aller Prozedurenkosten für jeden einzelnen Patienten. Falls dieser Wert nicht existiert, wird an ihrer Stelle 0 eingetragen. Jede Transaktion wird mit commt beendet. Das Speichern der neuen Einträge in die Faktentabelle wird überprüft

In [ ]:
cur.execute('''     UPDATE facts_table SET procedures_BASE_COST = proc.TOTALCOST
                    FROM
                    (SELECT PATIENT, SUM(BASE_COST) as TOTALCOST
                    FROM procedures, facts_table
                    WHERE facts_table.patient_ID = procedures.PATIENT
                    GROUP BY PATIENT) as proc
                    WHERE facts_table.patient_ID = proc.PATIENT
                    ;''')
conn.commit()

In [ ]:
cur.execute('''UPDATE
                  facts_table
               SET
                  procedures_BASE_COST = 0
                WHERE procedures_BASE_COST IS NULL
                    ;''')
conn.commit()

In [ ]:
cur.execute('''SELECT patient_ID, procedures_Base_COST, SUM(Base_COST)
                FROM facts_table, procedures WHERE facts_table.patient_ID = procedures.PATIENT
                GROUP BY patient_ID''')
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)

Die Daten werden aus **Immunizations** eintragen. Es handelt sich um die Summe aller Immunizationskosten für jeden einzelnen Patienten. Falls dieser Wert nicht existiert, wird an ihrer Stelle 0 eingetragen. Jede Transaktion wird mit commt beendet. Das Speichern der neuen Einträge in die Faktentabelle wird überprüft

In [ ]:
cur.execute('''     UPDATE facts_table SET immunizations_Base_Cost = im.TOTALCOST
                    FROM
                    (SELECT PATIENT, SUM(Base_Cost) as TOTALCOST
                    FROM immunizations, facts_table
                    WHERE facts_table.patient_ID = immunizations.PATIENT
                    GROUP BY PATIENT) as im
                    WHERE facts_table.patient_ID = im.PATIENT
                    ;''')
conn.commit()

In [ ]:
cur.execute('''UPDATE
                  facts_table
               SET
                  immunizations_BASE_COST = 0
                WHERE immunizations_BASE_COST IS NULL
                    ;''')
conn.commit()

In [ ]:
cur.execute(''' SELECT patient_ID, immunizations_BASE_COST, SUM(BASE_COST) FROM facts_table, immunizations
                WHERE facts_table.patient_ID = immunizations.PATIENT
                GROUP BY patient_ID''' )
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)

> Werte in der Faktentabelle haben Komma statt Dot für die Dezimale. Konvertierung dieser.


In [ ]:
cur.execute(''' UPDATE facts_table SET
patient_LAT = REPLACE(patient_LAT,',','.'),
patient_LON = REPLACE(patient_LON,',','.'),
patient_HEALTHCARE_EXPENSES = REPLACE(patient_HEALTHCARE_EXPENSES,',','.'),
patient_HEALTHCARE_COVERAGE = REPLACE(patient_HEALTHCARE_COVERAGE,',','.')
;''')
conn.commit()

##Das Datawarehouse ist fertig ausgefüllt.
Hier werden die ersten Tupeln der Faktentabelle angezeigt.
Der "None"-Eintrag entspricht der Spalte für Encounters. Diese Spalte wird nicht benutzt, deshalb wurde sie nicht ausgefüllt.

In [ ]:
cur.execute('''SELECT * FROM facts_table
 ORDER BY RANDOM()''')
records = cur.fetchall()
# show 4 rows
for row in records[:30]:
  print(row)

In [ ]:
print(pd.read_sql_query("PRAGMA table_info('facts_table')", conn))

# Bereinigung der Faktentabelle

## Delele all rows with Age = 0

In [ ]:
cur.execute('''SELECT COUNT(*)  FROM facts_table
 WHERE observations_AGE == 0''')
result = cur.fetchall()
print(f"Number of rows where observations_AGE == 0 : {result[-1][-1]}")
print("\n")

cur.execute(''' DELETE FROM facts_table WHERE observations_AGE = 0
;''')
conn.commit()

cur.execute('''SELECT COUNT(*)  FROM facts_table
 WHERE observations_AGE == 0''')
result = cur.fetchall()
print(f"Nach Bereinigung: Number of rows where observations_AGE == 0 : {result[-1][-1]}")
print("\n")


##Close Database connection

In [ ]:
conn.close()
print("The Sqlite connection is closed")

The Sqlite connection is closed
